In [1]:
import json
import httpx
import requests
import asyncio
import nest_asyncio
nest_asyncio.apply()

url = "http://localhost:8000"
client = httpx.AsyncClient(timeout=httpx.Timeout(300.0))

def invoke_api(s_id=None):
    api_url = f"{url}/session"
    if s_id:
        api_url = f"{api_url}/{s_id}/memory"
    res = requests.post(api_url)
    return res.json()

In [2]:
async def stream_api(payload):
    async with client.stream(**payload) as res:
        async for msg in res.aiter_text():
            chunk = json.loads(msg)
            if "node" in chunk:
                yield ["content"]
            else:
                yield chunk
async def draw_messages(messages_agen):
    while msg := await anext(messages_agen, None):  # noqa: F821        
        print(msg)

In [3]:
 
session_id = invoke_api()["session_id"]
msg = ""

p1 = {
    "method": "POST",
    "url": f"{url}/session/{session_id}/messages",
    "json": {
        "msg": {"message": {"content": msg,"type": "text"}}
}
}

stream = stream_api(p1)
print(asyncio.run(draw_messages(stream)))

In [15]:
mem = invoke_api(session_id)